In [1]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys
import math
import numpy as np
sys.path.append('../pytools')
import d2l

d2l.gpu_mem_init()

2025-09-22 13:09:41.138446: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-22 13:09:41.201326: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-22 13:09:42.650457: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'): True


In [2]:
def pool2d(X, pool_size, mode='max'):
    h,w = pool_size
    y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            if mode == 'max':
                y[i,j].assign(tf.reduce_max(X[i:i+h, j:j+w]))
            elif mode == 'avg':
                y[i,j].assign(tf.reduce_mean(X[i:i+h, j:j+w]))
    return y

In [3]:
X = tf.constant([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

I0000 00:00:1758517783.617614 1844943 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30501 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:00:08.0, compute capability: 7.0


<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[4., 5.],
       [7., 8.]], dtype=float32)>

In [4]:
pool2d(X, (2,2), 'max')

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[4., 5.],
       [7., 8.]], dtype=float32)>

In [5]:
X = tf.reshape(tf.range(16, dtype=tf.float32), (1, 4, 4, 1))
pool2d = tf.keras.layers.MaxPool2D(pool_size=[2, 2])
pool2d(X)

2025-09-22 13:09:43.750690: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


<tf.Tensor: shape=(1, 2, 2, 1), dtype=float32, numpy=
array([[[[ 5.],
         [ 7.]],

        [[13.],
         [15.]]]], dtype=float32)>

In [8]:
paddings = tf.constant([[0,0],[1,0], [1,0],[0,0]])
X = tf.Variable(tf.constant([[1,2,3,4],[5,6,7,8]]))
X = tf.reshape(X, (1, X.shape[0], X.shape[1], 1))
X_padded = tf.pad(X, paddings, "CONSTANT")

In [20]:
X = tf.concat([X, X+1], 3)
paddings = tf.constant([[0, 0], [1,0], [1,0], [0,0]])
x_padded = tf.pad(X, paddings, 'CONSTANT')
pool2d = tf.keras.layers.MaxPool2D(pool_size=[3,3], padding='valid', strides=2)
pool2d(X_padded)

<tf.Tensor: shape=(1, 1, 2, 1), dtype=int32, numpy=
array([[[[6],
         [8]]]], dtype=int32)>